# Exercise 10: Approximate Inference in Graphical Models

## Introduction

In this practical exercise, we implement the Gibbs sampling and logic sampling approximate inference algorithms. 

We study the graphical model over the six binary variables $x_1,x_2,x_3,x_4,x_5,x_6$ given in Exercise 1 of the last set of exercises (see last exercise sheet). 

We first define the graph structure by a variable *parents* that lists the parent nodes of each node. Because list indices start at zero, we use indices $0,...,5$ to refer to the nodes $x_1,x_2,x_3,x_4,x_5,x_6$. We also define a variable *tables* that represents the conditional probability tables for all nodes as defined in the exercise sheet.

In [1]:
parents = [[],[0],[1],[2],[0,3],[0,4]]
tables = [[0.5],[0.6,0.3],[0.7,0.4],[0.2,0.5],[0.3,0.6,0.5,0.1],[0.5,0.1,0.3,0.4]]

As discussed in the lecture, the graphical model defines a joint probability distribution 

$$p(x_1,...,x_6) = \prod_{i=1}^6 p(x_i \mid pa(x_i))$$.

The function *joint_probability* computes this joint probability for a particular joint state of the six random variables. Here, the joint state is given by a list of six values $x_i \in \{0,1\}$. The function is based on the function *probability_factor*, which computes the probability $p(x_i \mid pa(x_i))$ for a certain node given the state of its parents. 


In [2]:
import numpy as np

def joint_probability(joint_state):
    joint_state = np.array(joint_state)
    p = 1;
    for i in range(0,len(joint_state)): 
        p = p*probability_factor(joint_state[i],joint_state[parents[i]],tables[i])
    return p

def probability_factor(node_state,parent_state,probability_table):
    index = 0
    for j in range(0,len(parent_state)):
        index = index + 2**j*int(parent_state[j])
    if node_state:
        return probability_table[index]
    else:
        return 1-probability_table[index] 

For example, the following call computes the probability $p(x_1=0,x_2=1,x_3=0,x_4=1,x_5=0,x_6=1)$:

In [3]:
print("Probability of joint state [0,1,0,1,0,1]: %f." % joint_probability([0,1,0,1,0,1]))

Probability of joint state [0,1,0,1,0,1]: 0.009000.


We also supply a function *node_conditional_probability*, which computes the conditional probability $p(x_i = 1 \mid x_1,...,x_{i-1},x_{i+1},...,x_n)$ of $x_i$ being true given a joint state of all other nodes. The argument *i* is the index of the variable for which to compute the conditional, note that again the index range is $0,...,5$ for the variables $x_1,...,x_6$. The argument *joint_state* is a list of states for all nodes, the element *joint_state[i]* is ignored. As discussed in the lecture, this probability can be computed efficiently by straightforward application of the definition of conditional probability.


In [4]:
def node_conditional_probability(i,joint_state):
    p0 = joint_probability(np.concatenate((joint_state[:i],[0],joint_state[i+1:]))) 
    p1 = joint_probability(np.concatenate((joint_state[:i],[1],joint_state[i+1:])))
    return p1/(p0+p1)

For example, the following call computes the probability $p(x_3 = 1 \mid x_1=0,x_2=1,x_4=1,x_5=0,x_6=1)$:

In [5]:
print("Conditional probability p(x3=1|x1=0,x2=1,x4=1,x5=0,x6=1): %f" % node_conditional_probability(2,[0,1,0,1,0,1]))

Conditional probability p(x3=1|x1=0,x2=1,x4=1,x5=0,x6=1): 0.625000


## Exercises

### Exercise 1:

#### 1.1: 

Implement a function *gibbs_sampling(evidence,K)* that generates K samples using the Gibbs sampling algorithm. 
The argument *evidence* is a list $[e_1,e_2,e_3,e_4,e_5,e_6]$ of integer values, where a value of $e_i = 0$ indicates 
evidence $x_i=0$, $e_i = 1$ indicates $x_i=1$, and $e_i = -1$ indicates no evidence on the variable $x_i$. 

#### 1.2:

Use the implemented algorithm to approximately solve the inference problem $p(x_3 \mid x_1=0, x_4=1)$ that we already solved exactly using the message passing algorithm in the last exercise session. That is, compute a set of samples by

*samples = gibbs_sampling(evidence,K)*

and approximate the conditional distribution over the query variable $x_3$ by computing the fraction of times the state of $x_3$ was 1 over all samples.

Reminder: the solution from message passing was $p(x_3 = 1 \mid x_1=0, x_4=1) \approx 0.73$. Your algorithm should converge to this solution. Run the algorithm for $K = 100,1000,10000,100000$ samples, and note how the approximation error changes as a function of K. 

### Exercise 2: 

Also implement logic sampling, and use it to approximately solve the same inference problem. 